**Clone deep-fake repository**

# **Hướng dẫn sử dụng:**

- file này chỉ chạy không thay đổi gì
- cần vào telegram để thao tác 
      /d(download), 
      /u(upload)
- các file upload lên để trong thư mục upload, json sẽ tạo 2 bản trong thư mục json_upload/co_dinh, json_upload/tam. Chỉ nên sử dụng trong thu mục tam, cái cố định chức năng giống sao lưu
- muốn tải file nào thì để json của file đó vào thư mục json_download

In [38]:
#@title
from IPython.display import clear_output
#Install the telegram bot framework
!pip install pyTelegramBotAPI
from datetime import datetime 
import time
import json
!pip install download
from download import download
clear_output()


In [39]:
#@title
!git clone https://github.com/daovannguyen/telegram_bot_1.git
%cd telegram_bot_1
clear_output()

**Load the deep fake libraries**

In [40]:
#@title
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
cpu = False
#generator, kp_detector = load_checkpoints(config_path='config/vox-adv-256.yaml', 
#                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-adv-cpk.pth.tar',cpu=cpu)

In [41]:
#@title
import time
import os
from tqdm import tqdm
import shutil
%cd /content


/content


In [42]:
#@title
def create_folder(path):
  try:
    shutil.rmtree(path)
  except:
    saf = 0
  os.mkdir(path)

In [43]:
#@title
def get_time():
  start = str(datetime.now())
  start = start.split('.')[0].replace(' ', '_').replace('-', '_').replace(':', '_')
  start = '_' + start 
  return '_date_' + start

In [44]:
#@title
def split_file(path):
  new_folder = '/content/1'
  create_folder(new_folder)
  size = 19*1024*1024


  name = path.split('/')[-1] + get_time()

  if os.path.isfile(path):
    number = int(os.path.getsize(path) / size + 1)
    with open(path, 'rb') as f:
      for j in tqdm(range(number)):
        x = f.read(size)
        n = os.path.join(new_folder, str(j).zfill(4))
        with open(n, 'wb') as h:
          h.write(x)
  return name

In [45]:
#@title
def upload_file(folder_path, chat_id, bot):
  file_ids = []
  arr = os.listdir(folder_path)
  arr.sort()
  for i in arr:
    n = os.path.join(folder_path, i)
    a = bot.send_document(chat_id=chat_id, data = open(n, 'rb'))
    file_ids.append(a.document.file_id)
  return file_ids

In [46]:
#@title
def upload_bot(message, bot, path_json):
  chat_id = message.chat.id
  bot.reply_to(message, "Start upload....")

  with open(path_json, 'r') as f:
    paths = json.load(f)
  arr = list(paths)
  
  for i in tqdm(arr):
    name = split_file(i)
    ids = upload_file('/content/1', chat_id, bot)

    info = {}
    info['name'] = name
    info['file_ids'] = ids

    db_path = os.path.join('/content/drive/MyDrive/tong_ket/upload_telegram_bot/json_uploaded/co_dinh', name + '.json')
    with open(db_path, 'w') as f:
      json.dump(info, f, ensure_ascii=False)

    db_path = os.path.join('/content/drive/MyDrive/tong_ket/upload_telegram_bot/json_uploaded/tam', name + '.json')
    with open(db_path, 'w') as f:
      json.dump(info, f, ensure_ascii=False)
    bot.send_document(chat_id=chat_id, data = open(db_path, 'rb'))
    os.remove(i)
    bot.reply_to(message, name + '===========================')


    paths.remove(i)
    with open(path_json, 'w') as f:
      json.dump(paths, f, ensure_ascii=False)
    clear_output()

  bot.reply_to(message, 'End upload!')


In [47]:
#@title

def download_bot(bot, db_path, save):
  with open(db_path, 'r') as f:
    x = json.load(f)
  name = x['name'].split('_date')[0]
  ids = x['file_ids']
  path = os.path.join(save, name)
  try:
    os.remove(path)
  except:
    dsa = 0
  for i in tqdm(ids):
    file_info = bot.get_file(file_id=i)
    url = 'https://api.telegram.org/file/bot' + API_TOKEN + '/' + file_info.file_path
    file_local = 'dasdsad'
    download(url, file_local)
    with open(file_local, 'rb') as f:
      bytes_local = f.read()
    os.remove(file_local)
    with open(path, 'a+b') as f:
      f.write(bytes_local)
  os.remove(db_path)

In [48]:
#@title
def downloads_bot(bot, arr, save):
  for i in tqdm(arr):
    download_bot(bot, i, save)
    clear_output()

In [49]:
#@title
path_json = '/content/drive/MyDrive/tong_ket/upload_telegram_bot/json_uploaded/dung_trong_upload/tam.json'
folder_upload = '/content/drive/MyDrive/tong_ket/upload_telegram_bot/upload'
a = []
for i in os.listdir(folder_upload):
  n = os.path.join(folder_upload, i)
  a.append(n)
with open(path_json, 'w') as f:
  json.dump(a, f, ensure_ascii=False)

In [50]:
#@title
json_download = '/content/drive/MyDrive/tong_ket/upload_telegram_bot/json_download'
arr_json = []
for i in os.listdir(json_download):
  n = os.path.join(json_download, i)
  arr_json.append(n)
save = '/content/drive/MyDrive/tong_ket/upload_telegram_bot/download'

In [51]:
#@title
import telebot
import requests
import shutil
import subprocess
import os.path

API_TOKEN = '5087821904:AAGkgp-kNq98C3ghBvh9u5y8dGJjS5ZNhPY'
bot = telebot.TeleBot(API_TOKEN)
settings = {}
enabled = []

@bot.message_handler(commands=['u'])
def upload(message):
  upload_bot(message, bot, path_json)
  
@bot.message_handler(commands=['d'])
def download_a(message):
  bot.reply_to(message, 'Start download...')
  downloads_bot(bot, arr_json , save)
  bot.reply_to(message, 'End download!')


bot.polling()

100%|██████████| 1/1 [00:02<00:00,  2.08s/it]


In [52]:

function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
setInterval(ClickConnect, 60000)

SyntaxError: ignored